In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2015, 2023))


In [3]:
P_DATA_DIR = "Playoff_DATA"
P_STANDINGS_DIR = os.path.join(P_DATA_DIR, "p_standings")
#PLAYOFFS_DIR = os.path.join(DATA_DIR, "playoffs")
P_SCORES_DIR = os.path.join(P_DATA_DIR, "p_scores")

git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

In [4]:
#get html function
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout on {url}")
            continue
        else:
            break
    return html
    

In [5]:
# this function will scrape the playoff standings for each season
async def p_scrape_season(season):
    url = f"https://www.basketball-reference.com/playoffs/NBA_{season}_games.html"
    html = await get_html(url, "#info .button2.next")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    p_standings_pages = [f"https://www.basketball-reference.com{l}" for l in href ]

    for url in p_standings_pages:
        save_path = os.path.join(P_STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#div_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
# this function will scrape the playoff scores for each season
for season in SEASONS:
    await p_scrape_season(season)

2015 NBA Playoffs Schedule and Results | Basketball-Reference.com
2016 NBA Playoffs Schedule and Results | Basketball-Reference.com
2017 NBA Playoffs Schedule and Results | Basketball-Reference.com
2018 NBA Playoffs Schedule and Results | Basketball-Reference.com
2019 NBA Playoffs Schedule and Results | Basketball-Reference.com
2020 NBA Playoffs Schedule and Results | Basketball-Reference.com
2021 NBA Playoffs Schedule and Results | Basketball-Reference.com
2022 NBA Playoffs Schedule and Results | Basketball-Reference.com


In [7]:
standings_files = os.listdir(P_STANDINGS_DIR)

In [8]:
standings_files

['.DS_Store', 'p_scores', 'p_standings']

In [9]:
async def scrape_game(standings_files):
    with open(standings_files, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and l.startswith("/boxscores")]
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(P_SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        with open(save_path, "w+") as f:
            f.write(html)

In [10]:
standings_files = [s for s in standings_files if "html" in s]

In [11]:
for f in standings_files:
    filepath = os.path.join(P_DATA_DIR, f)
        
    await scrape_game(filepath)

In [12]:
standings_files

[]